In [ ]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score



In [ ]:
#viewing the dataset from my github repository
url="/content/heart_attack_russia_youth_vs_adult (1).csv"
df=pd.read_csv(url)
df.head()

,Age,Gender,Region,Blood_Pressure,Cholesterol,BMI,Heart_Rate,Exercise_Level,Smoking,Alcohol_Consumption,...,Heart_Attack,Angina,Heart_Disease_History,Diet,Sleep_Hours,Medication,Health_Awareness,Daily_Water_Intake,Mental_Health,Obesity
0,50,Male,Rural,110.0,196.5,15.9,76,High,False,Moderate,...,False,False,False,Unhealthy,9.4,False,5,2.3,5,False
1,40,Female,Urban,138.8,157.5,27.1,82,Moderate,False,NaN,...,False,True,False,Unhealthy,5.5,False,1,5.0,4,False
2,26,Male,Rural,116.0,210.1,27.2,71,Moderate,False,Moderate,...,False,False,False,Unhealthy,8.8,False,4,2.4,8,False
3,54,Female,Rural,133.5,170.5,26.0,74,Moderate,True,Moderate,...,False,False,False,Healthy,8.2,False,2,2.7,6,True
4,19,Female,Urban,108.0,224.5,27.5,67,Low,False,NaN,...,False,False,False,Unhealthy,5.9,False,4,3.5,4,True


In [ ]:
df_new = df.copy()
print("Initial null value counts:\n", df_new.isnull().sum())

for col in df_new.select_dtypes(include=['float']).columns:
    positive_values = df_new[col][df_new[col] > 0]
    if not positive_values.empty:
        mean_value = positive_values.mean()
        df_new[col] = df_new[col].apply(lambda x: mean_value if x <= 0 or pd.isnull(x) else x)

for column in df_new.select_dtypes(exclude='number').columns:
    df_new[column] = df_new[column].astype(str)

df_new = df_new.loc[:, ~df_new.columns.str.contains('^Unnamed')]
df_new = df_new.dropna()

print("\nFinal null value counts:\n", df_new.isnull().sum())

print("\nFinal DataFrame shape:", df_new.shape)

Initial null value counts:
 Age                          0
Gender                       0
Region                       0
Blood_Pressure               0
Cholesterol                  0
BMI                          0
Heart_Rate                   0
Exercise_Level               0
Smoking                      0
Alcohol_Consumption      25024
Diabetes                     0
Family_History               0
Stress_Level                 0
Heart_Attack                 0
Angina                       0
Heart_Disease_History        0
Diet                         0
Sleep_Hours                  0
Medication                   0
Health_Awareness             0
Daily_Water_Intake           0
Mental_Health                0
Obesity                      0
dtype: int64

Final null value counts:
 Age                      0
Gender                   0
Region                   0
Blood_Pressure           0
Cholesterol              0
BMI                      0
Heart_Rate               0
Exercise_Level           0
Smo

In [ ]:
# Identify categorical columns (excluding the target 'Heart_Attack')
categorical_columns = df_new.select_dtypes(include=['object']).columns.tolist()

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le

# Correlation matrix
correlation_matrix = df_new.corr()
correlation_matrix

,Age,Gender,Region,Blood_Pressure,Cholesterol,BMI,Heart_Rate,Exercise_Level,Smoking,Alcohol_Consumption,...,Heart_Attack,Angina,Heart_Disease_History,Diet,Sleep_Hours,Medication,Health_Awareness,Daily_Water_Intake,Mental_Health,Obesity
Age,1.000000,0.006217,0.000360,-0.000037,-0.005028,0.002640,-0.003697,0.012338,-0.001180,0.001938,...,0.000841,0.003647,0.004696,0.004960,-0.009355,-0.004225,0.006636,0.001802,-0.000388,0.004632
Gender,0.006217,1.000000,0.003473,-0.001385,0.000670,-0.001509,0.005610,-0.002092,-0.000642,0.001581,...,-0.002019,0.000502,0.000630,0.002848,-0.006928,-0.010958,-0.004616,0.004287,0.002471,0.001732
Region,0.000360,0.003473,1.000000,0.000274,-0.002419,0.001892,-0.002656,0.009957,-0.001593,0.002901,...,-0.003745,0.007911,-0.005312,-0.000111,-0.003595,0.003607,0.000250,-0.000070,0.000600,-0.001415
Blood_Pressure,-0.000037,-0.001385,0.000274,1.000000,0.001426,-0.004789,-0.012488,0.009455,-0.002458,-0.008163,...,-0.010560,0.009775,0.006524,0.003050,-0.001150,-0.006725,-0.003992,-0.001918,0.003967,0.006876
Cholesterol,-0.005028,0.000670,-0.002419,0.001426,1.000000,0.007163,0.002043,0.004223,-0.000226,-0.004555,...,-0.003460,-0.000882,-0.000067,0.004305,-0.008843,0.006578,0.001360,-0.000098,0.001024,0.004433
BMI,0.002640,-0.001509,0.001892,-0.004789,0.007163,1.000000,-0.009491,-0.003799,0.005467,0.005707,...,-0.000526,0.000228,0.001396,-0.001738,0.003105,-0.007269,-0.003134,-0.001232,0.000980,-0.009421
Heart_Rate,-0.003697,0.005610,-0.002656,-0.012488,0.002043,-0.009491,1.000000,0.004131,-0.011459,0.004087,...,-0.002241,0.003273,0.010129,0.002635,0.007489,-0.008292,-0.001228,-0.000040,-0.001706,0.004157
Exercise_Level,0.012338,-0.002092,0.009957,0.009455,0.004223,-0.003799,0.004131,1.000000,-0.005273,-0.000960,...,-0.000108,0.002229,-0.001870,0.000345,0.002983,0.005603,-0.002390,-0.000949,-0.001284,0.001542
Smoking,-0.001180,-0.000642,-0.001593,-0.002458,-0.000226,0.005467,-0.011459,-0.005273,1.000000,0.000739,...,0.010113,0.006129,0.001747,-0.005153,0.002280,0.000927,0.004412,-0.004509,0.003781,-0.001294
Alcohol_Consumption,0.001938,0.001581,0.002901,-0.008163,-0.004555,0.005707,0.004087,-0.000960,0.000739,1.000000,...,-0.002310,-0.001348,0.002449,0.002364,-0.004560,-0.002703,0.001574,-0.006140,-0.000389,0.009861


In [ ]:
# Define features and target variable
X = df_new.drop(columns=['Heart_Attack'])  # Adjust column name as per dataset
y = df_new['Heart_Attack']

In [ ]:
#!pip uninstall -y catboost numpy pandas
#!pip install catboost numpy==1.23.5 pandas==2.2.2 --no-cache-dir --force-reinstall

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: pandas 2.2.2
Uninstalling pandas-2.2.2:
  Successfully uninstalled pandas-2.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 278.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 301.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 288.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 132.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 212.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 311.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 352.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 274.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB

In [ ]:
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
def print_metrics(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.30, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train1)
X_test = scaler.transform(X_test1)

# Import necessary libraries
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight
from imblearn.ensemble import BalancedRandomForestClassifier
from xgboost import XGBClassifier
import numpy as np

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train1, y_train1)

# Compute class weights
classes = np.unique(y_train1)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train1)
class_weight_dict = dict(zip(classes, class_weights))

# CatBoost
cat_model1 = CatBoostClassifier(iterations=1000, learning_rate=0.05, depth=6, verbose=100, random_seed=42)
cat_model1.fit(X_train_resampled, y_train_resampled)
y_pred_cat1 = cat_model1.predict(X_test1)
print(f"CatBoost Accuracy: {accuracy_score(y_test1, y_pred_cat1):.4f}")
print_metrics("CatBoost", y_test1, y_pred_cat1)

# Gaussian Naive Bayes (No class_weight support)
gaus_model1 = GaussianNB()
gaus_model1.fit(X_train_resampled, y_train_resampled)
y_pred_gaus1 = gaus_model1.predict(X_test1)
print(f"Gaussian Naive Bayes Accuracy: {accuracy_score(y_test1, y_pred_gaus1):.4f}")
print_metrics("Gaussian Naive Bayes", y_test1, y_pred_gaus1)

# Random Forest with class weights
rf_model1 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=0, class_weight='balanced')
rf_model1.fit(X_train1, y_train1)
y_pred_rf1 = rf_model1.predict(X_test1)
print(f"Random Forest Accuracy: {accuracy_score(y_test1, y_pred_rf1):.4f}")
print_metrics("Random Forest", y_test1, y_pred_rf1)

# Balanced Random Forest
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=42)
brf_model.fit(X_train1, y_train1)
y_pred_brf = brf_model.predict(X_test1)
print(f"Balanced Random Forest Accuracy: {accuracy_score(y_test1, y_pred_brf):.4f}")
print_metrics("Balanced Random Forest", y_test1, y_pred_brf)

# Decision Tree with class weights
dt_model1 = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
dt_model1.fit(X_train1, y_train1)
y_pred_dt1 = dt_model1.predict(X_test1)
print(f"Decision Tree Accuracy: {accuracy_score(y_test1, y_pred_dt1):.4f}")
print_metrics("Decision Tree", y_test1, y_pred_dt1)

# K Nearest Neighbors (SMOTE applied)
knn_model1 = KNeighborsClassifier(n_neighbors=5)
knn_model1.fit(X_train_resampled, y_train_resampled)
y_pred_knn1 = knn_model1.predict(X_test1)
print(f"KNN Accuracy: {accuracy_score(y_test1, y_pred_knn1):.4f}")
print_metrics("KNN", y_test1, y_pred_knn1)

# Logistic Regression with class weights
lr_model1 = LogisticRegression(class_weight='balanced')
lr_model1.fit(X_train1, y_train1)
y_pred_lr1 = lr_model1.predict(X_test1)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test1, y_pred_lr1):.4f}")
print_metrics("Logistic Regression", y_test1, y_pred_lr1)

# Neural Network with SMOTE
nn_model = MLPClassifier(
    hidden_layer_sizes=(74, 103),
    activation='relu',
    solver='sgd',
    alpha=0.0020634128414587934,
    learning_rate_init=0.001374130105225485,
    max_iter=1000,
    random_state=42
)
nn_model.fit(X_train_resampled, y_train_resampled)
y_pred_nn = nn_model.predict(X_test1)
print(f"Neural Network Accuracy: {accuracy_score(y_test1, y_pred_nn):.4f}")
print_metrics("Neural Network", y_test1, y_pred_nn)


# AdaBoost (SMOTE applied)
ada_model1 = AdaBoostClassifier(n_estimators=50)
ada_model1.fit(X_train_resampled, y_train_resampled)
y_pred_ada1 = ada_model1.predict(X_test1)
print(f"AdaBoost Accuracy: {accuracy_score(y_test1, y_pred_ada1):.4f}")
print_metrics("AdaBoost", y_test1, y_pred_ada1)

# XGBoost with scale_pos_weight
xg_model1 = XGBClassifier(use_label_encoder=False, eval_metric='logloss', scale_pos_weight=class_weight_dict[1])
xg_model1.fit(X_train1, y_train1)
y_pred_xg1 = xg_model1.predict(X_test1)
print(f"XGBoost Accuracy: {accuracy_score(y_test1, y_pred_xg1):.4f}")
print_metrics("XGBoost", y_test1, y_pred_xg1)

# Gradient Boosting (SMOTE applied)
gb_model1 = GradientBoostingClassifier(n_estimators=50)
gb_model1.fit(X_train_resampled, y_train_resampled)
y_pred_gb1 = gb_model1.predict(X_test1)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_test1, y_pred_gb1):.4f}")
print_metrics("Gradient Boosting", y_test1, y_pred_gb1)

# Bagging (SMOTE applied)
bag_model1 = BaggingClassifier(n_estimators=50)
bag_model1.fit(X_train_resampled, y_train_resampled)
y_pred_bag1 = bag_model1.predict(X_test1)
print(f"Bagging Accuracy: {accuracy_score(y_test1, y_pred_bag1):.4f}")
print_metrics("Bagging", y_test1, y_pred_bag1)

# Another Neural Network (SMOTE applied)
nn_model1 = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', max_iter=500)
nn_model1.fit(X_train_resampled, y_train_resampled)
y_pred_nn1 = nn_model1.predict(X_test1)
print(f"Neural Network Accuracy: {accuracy_score(y_test1, y_pred_nn1):.4f}")
print_metrics("Neural Network", y_test1, y_pred_nn1)

0:	learn: 0.6792988	total: 145ms	remaining: 2m 25s
100:	learn: 0.4288413	total: 4.72s	remaining: 42s
200:	learn: 0.3598683	total: 8.61s	remaining: 34.2s
300:	learn: 0.3031956	total: 14.5s	remaining: 33.6s
400:	learn: 0.2741480	total: 19s	remaining: 28.4s
500:	learn: 0.2585820	total: 23.6s	remaining: 23.5s
600:	learn: 0.2436001	total: 29.4s	remaining: 19.5s
700:	learn: 0.2334252	total: 34.4s	remaining: 14.7s
800:	learn: 0.2239576	total: 36.4s	remaining: 9.04s
900:	learn: 0.2168291	total: 39.5s	remaining: 4.34s
999:	learn: 0.2094958	total: 41.1s	remaining: 0us
CatBoost Accuracy: 0.8747
CatBoost - Accuracy: 0.8747, Precision: 0.7961, Recall: 0.8747, F1 Score: 0.8278
Gaussian Naive Bayes Accuracy: 0.6088
Gaussian Naive Bayes - Accuracy: 0.6088, Precision: 0.7961, Recall: 0.6088, F1 Score: 0.6778
Random Forest Accuracy: 0.8394
Random Forest - Accuracy: 0.8394, Precision: 0.7961, Recall: 0.8394, F1 Score: 0.8158
Balanced Random Forest Accuracy: 0.8465
Balanced Random Forest - Accuracy: 0.846

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy: 0.5257
Logistic Regression - Accuracy: 0.5257, Precision: 0.7968, Recall: 0.5257, F1 Score: 0.6098
Neural Network Accuracy: 0.6802
Neural Network - Accuracy: 0.6802, Precision: 0.7960, Recall: 0.6802, F1 Score: 0.7284
AdaBoost Accuracy: 0.6535
AdaBoost - Accuracy: 0.6535, Precision: 0.7951, Recall: 0.6535, F1 Score: 0.7101


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:36:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBoost Accuracy: 0.8358
XGBoost - Accuracy: 0.8358, Precision: 0.7934, Recall: 0.8358, F1 Score: 0.8130
Gradient Boosting Accuracy: 0.7203
Gradient Boosting - Accuracy: 0.7203, Precision: 0.7949, Recall: 0.7203, F1 Score: 0.7535
Bagging Accuracy: 0.8061
Bagging - Accuracy: 0.8061, Precision: 0.7971, Recall: 0.8061, F1 Score: 0.8015
Neural Network Accuracy: 0.7433
Neural Network - Accuracy: 0.7433, Precision: 0.7953, Recall: 0.7433, F1 Score: 0.7672


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier

# Models
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

# Splitting dataset
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.30, random_state=42)

# Standardization
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train1, y_train1)

# Compute class weights
classes = np.unique(y_train1)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train1)
class_weight_dict = dict(zip(classes, class_weights))

### **Model Training and Evaluation**
# CatBoost (Handles imbalance natively)
cat_model = CatBoostClassifier(iterations=1000, learning_rate=0.05, depth=6, verbose=100, random_seed=42)
cat_model.fit(X_train_resampled, y_train_resampled)
y_pred_cat = cat_model.predict(X_test1)
print(f"CatBoost Accuracy: {accuracy_score(y_test1, y_pred_cat):.4f}")
print_metrics("CatBoost", y_test1, y_pred_cat)

# Gaussian Naive Bayes
gaus_model = GaussianNB()
gaus_model.fit(X_train_resampled, y_train_resampled)
y_pred_gaus = gaus_model.predict(X_test1)
print(f"Gaussian Naive Bayes Accuracy: {accuracy_score(y_test1, y_pred_gaus):.4f}")
print_metrics("Gaussian Naive Bayes", y_test1, y_pred_gaus)

# Random Forest with class weights
rf_model = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0, class_weight='balanced')
rf_model.fit(X_train1, y_train1)
y_pred_rf = rf_model.predict(X_test1)
print(f"Random Forest Accuracy: {accuracy_score(y_test1, y_pred_rf):.4f}")
print_metrics("Random Forest", y_test1, y_pred_rf)

# Balanced Random Forest
brf_model = BalancedRandomForestClassifier(n_estimators=1000, random_state=42)
brf_model.fit(X_train1, y_train1)
y_pred_brf = brf_model.predict(X_test1)
print(f"Balanced Random Forest Accuracy: {accuracy_score(y_test1, y_pred_brf):.4f}")
print_metrics("Balanced Random Forest", y_test1, y_pred_brf)

# Decision Tree with class weights
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
dt_model.fit(X_train1, y_train1)
y_pred_dt = dt_model.predict(X_test1)
print(f"Decision Tree Accuracy: {accuracy_score(y_test1, y_pred_dt):.4f}")
print_metrics("Decision Tree", y_test1, y_pred_dt)

# KNN (SMOTE applied)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_resampled, y_train_resampled)
y_pred_knn = knn_model.predict(X_test1)
print(f"KNN Accuracy: {accuracy_score(y_test1, y_pred_knn):.4f}")
print_metrics("KNN", y_test1, y_pred_knn)

# Logistic Regression with class weights
lr_model = LogisticRegression(class_weight='balanced', max_iter=1000)
lr_model.fit(X_train1, y_train1)
y_pred_lr = lr_model.predict(X_test1)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test1, y_pred_lr):.4f}")
print_metrics("Logistic Regression", y_test1, y_pred_lr)

# Neural Network (SMOTE applied)
nn_model = MLPClassifier(
    hidden_layer_sizes=(74, 103),
    activation='relu',
    solver='sgd',
    alpha=0.0020634128414587934,
    learning_rate_init=0.001374130105225485,
    max_iter=1000,
    random_state=42
)
nn_model.fit(X_train_resampled, y_train_resampled)
y_pred_nn = nn_model.predict(X_test1)
print(f"Neural Network Accuracy: {accuracy_score(y_test1, y_pred_nn):.4f}")
print_metrics("Neural Network", y_test1, y_pred_nn)

# SVM with class weights
svm_model = SVC(class_weight='balanced')
svm_model.fit(X_train1, y_train1)
y_pred_svm = svm_model.predict(X_test1)
print(f"SVM Accuracy: {accuracy_score(y_test1, y_pred_svm):.4f}")
print_metrics("SVM", y_test1, y_pred_svm)

# AdaBoost (SMOTE applied)
ada_model = AdaBoostClassifier(n_estimators=1000)
ada_model.fit(X_train_resampled, y_train_resampled)
y_pred_ada = ada_model.predict(X_test1)
print(f"AdaBoost Accuracy: {accuracy_score(y_test1, y_pred_ada):.4f}")
print_metrics("AdaBoost", y_test1, y_pred_ada)

# XGBoost with scale_pos_weight
scale_pos_weight = class_weight_dict.get(1, 1)  # Handle missing class 1 dynamically
xg_model = XGBClassifier(eval_metric='logloss', scale_pos_weight=scale_pos_weight, random_state=42)
xg_model.fit(X_train1, y_train1)
y_pred_xg = xg_model.predict(X_test1)
print(f"XGBoost Accuracy: {accuracy_score(y_test1, y_pred_xg):.4f}")
print_metrics("XGBoost", y_test1, y_pred_xg)

# Gradient Boosting (SMOTE applied)
gb_model = GradientBoostingClassifier(n_estimators=1000)
gb_model.fit(X_train_resampled, y_train_resampled)
y_pred_gb = gb_model.predict(X_test1)
print(f"Gradient Boosting Accuracy: {accuracy_score(y_test1, y_pred_gb):.4f}")
print_metrics("Gradient Boosting", y_test1, y_pred_gb)

# Bagging (SMOTE applied)
bag_model = BaggingClassifier(n_estimators=1000)
bag_model.fit(X_train_resampled, y_train_resampled)
y_pred_bag = bag_model.predict(X_test1)
print(f"Bagging Accuracy: {accuracy_score(y_test1, y_pred_bag):.4f}")
print_metrics("Bagging", y_test1, y_pred_bag)

# Another Neural Network (SMOTE applied)
nn_model1 = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', max_iter=500, random_state=42)
nn_model1.fit(X_train_resampled, y_train_resampled)
y_pred_nn1 = nn_model1.predict(X_test1)
print(f"Neural Network Accuracy: {accuracy_score(y_test1, y_pred_nn1):.4f}")
print_metrics("Neural Network", y_test1, y_pred_nn1)

0:	learn: 0.6519520	total: 80.5ms	remaining: 1m 20s
100:	learn: 0.2396288	total: 5.8s	remaining: 51.6s
200:	learn: 0.2204122	total: 10.6s	remaining: 42.1s
300:	learn: 0.2084741	total: 17.1s	remaining: 39.6s
400:	learn: 0.2016335	total: 22s	remaining: 32.9s
500:	learn: 0.1959739	total: 27.8s	remaining: 27.7s
600:	learn: 0.1908175	total: 32.7s	remaining: 21.7s
700:	learn: 0.1862110	total: 34.8s	remaining: 14.8s
800:	learn: 0.1817220	total: 38.3s	remaining: 9.52s
900:	learn: 0.1772380	total: 40.4s	remaining: 4.44s
999:	learn: 0.1729748	total: 42.5s	remaining: 0us
CatBoost Accuracy: 0.8841
CatBoost - Accuracy: 0.8841, Precision: 0.7818, Recall: 0.8841, F1 Score: 0.8298
Gaussian Naive Bayes Accuracy: 0.5623
Gaussian Naive Bayes - Accuracy: 0.5623, Precision: 0.7961, Recall: 0.5623, F1 Score: 0.6411
Random Forest Accuracy: 0.8589
Random Forest - Accuracy: 0.8589, Precision: 0.7950, Recall: 0.8589, F1 Score: 0.8227
Balanced Random Forest Accuracy: 0.8787
Balanced Random Forest - Accuracy: 0.8

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


Neural Network Accuracy: 0.7689
Neural Network - Accuracy: 0.7689, Precision: 0.7960, Recall: 0.7689, F1 Score: 0.7818
SVM Accuracy: 0.5840
SVM - Accuracy: 0.5840, Precision: 0.7961, Recall: 0.5840, F1 Score: 0.6586
AdaBoost Accuracy: 0.8842
AdaBoost - Accuracy: 0.8842, Precision: 0.7818, Recall: 0.8842, F1 Score: 0.8299


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBoost Accuracy: 0.8358
XGBoost - Accuracy: 0.8358, Precision: 0.7934, Recall: 0.8358, F1 Score: 0.8130
Gradient Boosting Accuracy: 0.8835
Gradient Boosting - Accuracy: 0.8835, Precision: 0.7914, Recall: 0.8835, F1 Score: 0.8297
Bagging Accuracy: 0.8839
Bagging - Accuracy: 0.8839, Precision: 0.7818, Recall: 0.8839, F1 Score: 0.8297
Neural Network Accuracy: 0.7486
Neural Network - Accuracy: 0.7486, Precision: 0.7966, Recall: 0.7486, F1 Score: 0.7708


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE

# Models
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# Load and split dataset (Replace with actual dataset)
#df = pd.read_csv("/content/heart_attack_russia_youth_vs_adult (1).csv")
# Define features and target variable
X = df_new.drop(columns=['Heart_Attack'])  # Adjust column name as per dataset
y = df_new['Heart_Attack']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# Apply SMOTE for KNN, Logistic Regression, and SVM (since they don’t support class_weight)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train1, y_train1)

# Compute class weights (for Decision Tree and Logistic Regression)
classes = np.unique(y_train1)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train1)
class_weight_dict = dict(zip(classes, class_weights))

# Standardize features for KNN, Logistic Regression, and SVM
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train1)
X_test_scaled = scaler.transform(X_test1)

X_train_resampled_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled_resampled = scaler.transform(X_test1)

# ------------------ Gaussian Naive Bayes ------------------ #
gnb_model = GaussianNB()
gnb_model.fit(X_train1, y_train1)
y_pred_gnb = gnb_model.predict(X_test1)
print(f"Gaussian Naive Bayes Accuracy: {accuracy_score(y_test1, y_pred_gnb):.4f}")

# ------------------ Decision Tree (Tuned) ------------------ #
param_grid_dt = {
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'criterion': ['gini', 'entropy']
}
dt_grid = GridSearchCV(DecisionTreeClassifier(random_state=42, class_weight=class_weight_dict),
                       param_grid_dt, cv=5, scoring='accuracy', n_jobs=-1)
dt_grid.fit(X_train1, y_train1)

best_dt_model = dt_grid.best_estimator_
y_pred_dt = best_dt_model.predict(X_test1)
print(f"Decision Tree Optimized Accuracy: {accuracy_score(y_test1, y_pred_dt):.4f}")

# ------------------ K-Nearest Neighbors (Tuned) ------------------ #
param_grid_knn = {'n_neighbors': [3, 5, 7, 9, 11]}
knn_grid = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring='accuracy', n_jobs=-1)
knn_grid.fit(X_train_resampled_scaled, y_train_resampled)

best_knn_model = knn_grid.best_estimator_
y_pred_knn = best_knn_model.predict(X_test_scaled_resampled)
print(f"KNN Optimized Accuracy: {accuracy_score(y_test1, y_pred_knn):.4f}")

# ------------------ Logistic Regression (Tuned) ------------------ #
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['liblinear', 'lbfgs']
}
lr_grid = GridSearchCV(LogisticRegression(class_weight='balanced', max_iter=1000),
                       param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
lr_grid.fit(X_train_resampled_scaled, y_train_resampled)

best_lr_model = lr_grid.best_estimator_
y_pred_lr = best_lr_model.predict(X_test_scaled_resampled)
print(f"Logistic Regression Optimized Accuracy: {accuracy_score(y_test1, y_pred_lr):.4f}")

from sklearn.metrics import precision_score, recall_score, f1_score

# Function to print evaluation metrics
def print_metrics(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

# Gaussian Naive Bayes
print_metrics("Gaussian Naive Bayes", y_test1, y_pred_gnb)

# Decision Tree (Tuned)
print_metrics("Decision Tree Optimized", y_test1, y_pred_dt)

# K-Nearest Neighbors (Tuned)
print_metrics("KNN Optimized", y_test1, y_pred_knn)

# Logistic Regression (Tuned)
print_metrics("Logistic Regression Optimized", y_test1, y_pred_lr)


Gaussian Naive Bayes Accuracy: 0.8824
Decision Tree Optimized Accuracy: 0.7929
KNN Optimized Accuracy: 0.6586
Logistic Regression Optimized Accuracy: 0.6851
Gaussian Naive Bayes - Accuracy: 0.8824, Precision: 0.7786, Recall: 0.8824, F1 Score: 0.8273
Decision Tree Optimized - Accuracy: 0.7929, Precision: 0.7925, Recall: 0.7929, F1 Score: 0.7927
KNN Optimized - Accuracy: 0.6586, Precision: 0.7949, Recall: 0.6586, F1 Score: 0.7131
Logistic Regression Optimized - Accuracy: 0.6851, Precision: 0.7937, Recall: 0.6851, F1 Score: 0.7307


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.ensemble import BalancedRandomForestClassifier

# Models
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

# Splitting dataset
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.25, random_state=42)

# Standardization
scaler = StandardScaler()
X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)

# Apply SMOTE for oversampling
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train1, y_train1)

# Compute class weights
classes = np.unique(y_train1)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train1)
class_weight_dict = dict(zip(classes, class_weights))

### **Model Training**
# Gaussian Naive Bayes
gaus_model = GaussianNB()
gaus_model.fit(X_train_resampled, y_train_resampled)

# Balanced Random Forest
brf_model = BalancedRandomForestClassifier(n_estimators=500, random_state=42)
brf_model.fit(X_train_resampled, y_train_resampled)

# Decision Tree
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42, class_weight='balanced')
dt_model.fit(X_train_resampled, y_train_resampled)

# AdaBoost
ada_model = AdaBoostClassifier(n_estimators=500, random_state=42)
ada_model.fit(X_train_resampled, y_train_resampled)

# Gradient Boosting
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train_resampled, y_train_resampled)

# XGBoost
xg_model = XGBClassifier(eval_metric='logloss', random_state=42)
xg_model.fit(X_train_resampled, y_train_resampled)

# CatBoost
cat_model = CatBoostClassifier(iterations=500, learning_rate=0.05, depth=6, verbose=100, random_seed=42)
cat_model.fit(X_train_resampled, y_train_resampled)

# Bagging
bag_model = BaggingClassifier(n_estimators=500, random_state=42)
bag_model.fit(X_train_resampled, y_train_resampled)

### **Voting Classifier (Majority Vote)**
voting_clf = VotingClassifier(
    estimators=[
        ('gnb', gaus_model),
        ('brf', brf_model),
        ('dt', dt_model),
        ('adaboost', ada_model),
        ('grad_boost', gb_model),
        ('xgb', xg_model),
        ('catboost', cat_model),
        ('bagging', bag_model)
    ],
    voting='hard'  # Hard voting selects the majority class
)

# Train the voting classifier
voting_clf.fit(X_train_resampled, y_train_resampled)

# Predict using the voting classifier
y_pred_voting = voting_clf.predict(X_test1)

# Print accuracy and performance metrics
def print_metrics(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    print(f"{model_name} Accuracy: {accuracy:.4f}")

print_metrics("Voting Classifier", y_test1, y_pred_voting)


0:	learn: 0.6518863	total: 69ms	remaining: 34.4s
100:	learn: 0.2387984	total: 2.02s	remaining: 7.97s
200:	learn: 0.2216464	total: 3.96s	remaining: 5.89s
300:	learn: 0.2102924	total: 6s	remaining: 3.96s
400:	learn: 0.2038898	total: 8.92s	remaining: 2.2s
499:	learn: 0.1986948	total: 10.7s	remaining: 0us
0:	learn: 0.6518863	total: 23.3ms	remaining: 11.6s
100:	learn: 0.2387984	total: 1.98s	remaining: 7.82s
200:	learn: 0.2216464	total: 3.93s	remaining: 5.84s
300:	learn: 0.2102924	total: 5.94s	remaining: 3.93s
400:	learn: 0.2038898	total: 7.86s	remaining: 1.94s
499:	learn: 0.1986948	total: 9.83s	remaining: 0us
Voting Classifier Accuracy: 0.8864


In [ ]:
def print_metrics1(model_name, y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"{model_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
print_metrics1("Voting Classifier", y_test1, y_pred_voting)

Voting Classifier - Accuracy: 0.8864, Precision: 0.7857, Recall: 0.8864, F1 Score: 0.8330


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
categorical_columns = df_new.select_dtypes(include=['object']).columns.tolist()

label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df_new[col] = le.fit_transform(df_new[col])
    label_encoders[col] = le


In [ ]:
# Define a test sample (Replace with actual feature values)
test_sample = np.array([[32,'Female','Rural',104.3,257.4,17.6,60,'Low','TRUE','None','TRUE','FALSE',10]])  # Example input

# Ensure the test sample is standardized using the same scaler
test_sample = scaler.transform(test_sample)

# Predict using the trained voting classifier
predicted_label = voting_clf.predict(test_sample)

# Interpret the result
result = "Prone to Heart Attack" if predicted_label[0] == 1 else "Not Prone to Heart Attack"
print(f"Test Sample Prediction: {result}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


ValueError: could not convert string to float: 'Female'

In [ ]:
import numpy as np
import pandas as pd

# Ensure all float columns have valid values
df_new = df.copy()
print("Initial null value counts:\n", df_new.isnull().sum())

# Handle negative and missing values in float columns
for col in df_new.select_dtypes(include=['float']).columns:
    positive_values = df_new[col][df_new[col] > 0]
    if not positive_values.empty:
        mean_value = positive_values.mean()
        df_new[col] = df_new[col].apply(lambda x: mean_value if x <= 0 or pd.isnull(x) else x)

# Convert non-numeric columns to string
for column in df_new.select_dtypes(exclude='number').columns:
    df_new[column] = df_new[column].astype(str)

# Drop unnecessary columns
df_new = df_new.loc[:, ~df_new.columns.str.contains('^Unnamed')]
df_new = df_new.dropna()

print("\nFinal null value counts:\n", df_new.isnull().sum())
print("\nFinal DataFrame shape:", df_new.shape)

# ----------------- TEST SAMPLE PROCESSING ----------------- #

# Define a test sample (Replace with actual feature values)
test_sample = np.array([['32', 'Female', 'Rural', 104.3, 257.4, 17.6, 60, 'Low', 'TRUE', 'None', 'TRUE', 'FALSE', 10,
                         'FALSE', 'FALSE', 'Healthy', 9.9, 'TRUE', 1, 1.3, 7, 'FALSE']], dtype=object)

# Convert categorical values using the saved label encoders
for i, col in enumerate(categorical_columns):
    if col in label_encoders:  # Check if the column was encoded in training
        try:
            # Encode known values
            test_sample[0, i] = label_encoders[col].transform([test_sample[0, i]])[0]
        except ValueError:
            # Handle unseen values: replace with the most frequent category
            most_common_category = label_encoders[col].classes_[0]  # Get first category from trained encoder
            test_sample[0, i] = label_encoders[col].transform([most_common_category])[0]

# Convert all string values to float properly
try:
    test_sample = np.array(test_sample, dtype=float)
except ValueError as e:
    print(f"Error in conversion to float: {e}")
    print(f"Problematic data: {test_sample}")

    # Try a second approach by iterating and converting elements one by one
    for i in range(test_sample.shape[1]):
        try:
            test_sample[0, i] = float(test_sample[0, i])
        except ValueError:
            print(f"Still unable to convert column {i} with value: {test_sample[0, i]}")
            test_sample[0, i] = 0.0  # Replace non-convertible values with 0.0

# Ensure the test sample is standardized using the same scaler
test_sample_scaled = scaler.transform(test_sample.reshape(1, -1))  # Reshape for transformation

# Predict using the trained voting classifier
predicted_label = voting_clf.predict(test_sample_scaled)

# Interpret the result
result = "Prone to Heart Attack" if predicted_label[0] == 1 else "Not Prone to Heart Attack"
print(f"Test Sample Prediction: {result}")


Initial null value counts:
 Age                          0
Gender                       0
Region                       0
Blood_Pressure               0
Cholesterol                  0
BMI                          0
Heart_Rate                   0
Exercise_Level               0
Smoking                      0
Alcohol_Consumption      25024
Diabetes                     0
Family_History               0
Stress_Level                 0
Heart_Attack                 0
Angina                       0
Heart_Disease_History        0
Diet                         0
Sleep_Hours                  0
Medication                   0
Health_Awareness             0
Daily_Water_Intake           0
Mental_Health                0
Obesity                      0
dtype: int64

Final null value counts:
 Age                      0
Gender                   0
Region                   0
Blood_Pressure           0
Cholesterol              0
BMI                      0
Heart_Rate               0
Exercise_Level           0
Smo

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Test Sample Prediction: Not Prone to Heart Attack


voting code with better prediction

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Fix for CatBoost inside VotingClassifier
class SklearnCompatibleCatBoost(CatBoostClassifier):
    def fit(self, X, y):
        return super().fit(Pool(X, label=y), verbose=0)

# Define models with best parameters
models = [
    ('gnb', GaussianNB()),  # Gaussian Naïve Bayes
    ('rf', RandomForestClassifier(n_estimators=500, random_state=42)),  # Balanced Random Forest
    ('dt', DecisionTreeClassifier(random_state=42)),  # Decision Tree
    ('ada', AdaBoostClassifier(n_estimators=500, random_state=42)),  # AdaBoost
    ('gb', GradientBoostingClassifier(n_estimators=100, random_state=42)),  # Gradient Boosting (Updated)
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),  # XGBoost
    ('cat', SklearnCompatibleCatBoost(iterations=500, random_state=42)),  # Fixed CatBoost
    ('bag', BaggingClassifier(n_estimators=500, random_state=42))  # Bagging (Updated)
]

# Create a Voting Classifier (Soft Voting for probabilistic averaging)
voting_clf = VotingClassifier(estimators=models, voting='soft')

# Train the ensemble model
voting_clf.fit(X_train_resampled_scaled, y_train_resampled)

# Make predictions
y_pred = voting_clf.predict(X_test_scaled_resampled)

# Evaluate performance
print("Accuracy:", accuracy_score(y_test1, y_pred))
print("Classification Report:\n", classification_report(y_test1, y_pred))

print_metrics("Logistic Regression Optimized", y_test1, y_pred)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:47:23] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Accuracy: 0.7996
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.89      0.89     13236
           1       0.13      0.13      0.13      1764

    accuracy                           0.80     15000
   macro avg       0.51      0.51      0.51     15000
weighted avg       0.80      0.80      0.80     15000

Logistic Regression Optimized - Accuracy: 0.7996, Precision: 0.7960, Recall: 0.7996, F1 Score: 0.7978


In [ ]:
import numpy as np

# Define the test sample (raw values)
test_sample = {
    "Age": 53,
    "Gender": "Female",
    "Region": "Urban",
    "Blood_Pressure": 115.9,
    "Cholesterol": 231.9,
    "BMI": 32,
    "Heart_Rate": 76,
    "Exercise_Level": "Moderate",
    "Smoking": "False",
    "Alcohol_Consumption": "Moderate",
    "Diabetes": "False",
    "Family_History": "False",
    "Stress_Level": 1,
    "Angina": "False",
    "Heart_Disease_History": "False",
    "Diet": "Mixed",
    "Sleep_Hours": 4.2,
    "Medication": "True",
    "Health_Awareness": 4,
    "Daily_Water_Intake": 3.4,
    "Mental_Health": 2,
    "Obesity": "False"
}

# Encoding mappings from your dataset
encoding_map = {
    'Gender': {'Female': 0, 'Male': 1, 'Other': 2},
    'Region': {'Rural': 0, 'Suburban': 1, 'Urban': 2},
    'Exercise_Level': {'High': 0, 'Low': 1, 'Moderate': 2},
    'Smoking': {'False': 0, 'True': 1},
    'Alcohol_Consumption': {'Heavy': 0, 'Moderate': 1, 'None': 2},
    'Diabetes': {'False': 0, 'True': 1},
    'Family_History': {'False': 0, 'True': 1},
    'Angina': {'False': 0, 'True': 1},
    'Heart_Disease_History': {'False': 0, 'True': 1},
    'Diet': {'Healthy': 0, 'Mixed': 1, 'Unhealthy': 2},
    'Medication': {'False': 0, 'True': 1},
    'Obesity': {'False': 0, 'True': 1}
}

# Apply encoding to categorical and boolean values
encoded_sample = {}
for feature, value in test_sample.items():
    if feature in encoding_map:  # If categorical or boolean, encode it
        encoded_sample[feature] = encoding_map[feature][value]
    else:  # If numerical, keep as is
        encoded_sample[feature] = value

# Convert dictionary to NumPy array (reshape for model input)
test_sample_array = np.array(list(encoded_sample.values())).reshape(1, -1)

# Standardize test sample using the same scaler used in training
test_sample_scaled = scaler.transform(test_sample_array)

# Predict probability for the test sample
test_pred_proba = voting_clf.predict_proba(test_sample_scaled)[0]  # Get probability for both classes

# Display the probability of heart attack risk
print("\n🔍 **Test Sample Prediction:**")
print(f"Chance of NO Heart Attack: {test_pred_proba[0] * 100:.2f}%")
print(f"Chance of Heart Attack: {test_pred_proba[1] * 100:.2f}%")

# Predict class label
test_pred_label = voting_clf.predict(test_sample_scaled)[0]
print(f"\nFinal Prediction: {'Heart Attack' if test_pred_label == 1 else 'No Heart Attack'}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(



🔍 **Test Sample Prediction:**
Chance of NO Heart Attack: 52.34%
Chance of Heart Attack: 47.66%

Final Prediction: No Heart Attack
